In [1]:
import numpy as np
import pandas as pd
import jieba as jb
import matplotlib.pyplot as plt
import re

In [2]:
data = pd.read_excel('data.xlsx')

In [3]:
# 添加了参考法律和条目
law = []
line = []
for index in range(len(data)):
    #print(data.iloc[index]["判决结果"])
    try:
        # 提取法律名称
        law.append(re.search(r'《.*?》',data.iloc[index]["判决结果"]).group(0))
        # 提取所用法条
        line.append(re.search(r'第.*?条',data.iloc[index]["判决结果"]).group(0))
    except:
        law.append(np.nan)
        line.append(np.nan)

data['判决法律'] = pd.DataFrame(law)
data['判决法条'] = pd.DataFrame(line)

In [4]:
final = []
for index in range(len(data)):        
    # 提取是否为终审
    try:
        if "终审" in data.iloc[index]['庭后告知']:
            final.append(1)
        else:
            final.append(0)
    except:
        final.append(0)
data['是否为终审'] = pd.DataFrame(final)
del data['庭后告知']
del data['庭后告知2']

In [82]:
del data['结尾']
del data['结尾2']

# delete useless lines
del data['庭审过程6']
del data['庭审过程3']
del data['庭审过程4']
del data['庭审过程5']

del data['头部2']
del data['当事人2']
del data['庭审程序说明2']
del data['法院意见2']

del data['判决结果2']
del data['庭审过程2']

#标题只是在复述中间的信息，可以删除
del data['标题']

del data['附录2']
del data['法官'] # 假设于法官、审判长无关，假设与日期无关，假设与书记员、审判员无关
del data['审判长']
del data['书记员']
del data['审判员']

del data['案号']
del data['案件类型']

# 与什么法院，似乎没有特别大关系。有可能大城市更容易倾向个人？而小城市倾向大企业？这里不做考虑
del data['法院']
# 与判决日期的关系 也不大
del data['判决日期']

# 头部就是所有信息的综合 删去
del data['头部']

KeyError: '结尾'

In [6]:
# 公司告公司 为1；人告公司 为2；公司告人 为3；其他 为4 
sue_type = []
for index in range(len(data)):        
    try:
        if "公司" in data.iloc[index]['原告'] and "公司" in data.iloc[index]['被告']:
            sue_type.append(1)
        elif "公司" not in data.iloc[index]['原告'] and "公司" in data.iloc[index]['被告']:
            sue_type.append(2)
        elif "公司" in data.iloc[index]['原告'] and "公司" not in data.iloc[index]['被告']:
            sue_type.append(3)
        else:
            sue_type.append(4)
    except:
        sue_type.append(0)
data['原被告分类'] = pd.DataFrame(sue_type)
#data.iloc[0]['原被告分类'] = 2
del data['原告']
del data['被告']

In [8]:
data.count()

庭审程序      1000
案由        1000
文书类型      1000
第三人         62
当事人        998
庭审程序说明    1000
庭审过程       795
法院意见       849
判决结果       937
附录         166
判决法律       931
判决法条       931
是否为终审     1000
原被告分类     1000
dtype: int64

In [9]:
# 处理第三方
# 第三方为公司 1； 第三方为人 2； 无第三方 0 
thirdparty = []
for index in range(len(data)):        
    try:
        if "公司" in data.iloc[index]['第三人']:
            thirdparty.append(1)
        elif "公司" not in data.iloc[index]['第三人']:
            thirdparty.append(2)
    except:
        thirdparty.append(0)

data['第三方分类'] = pd.DataFrame(thirdparty)
del data['第三人']

In [10]:
anyou = data['案由']
anyoulist = []

for i in range(len(anyou)):
    if anyou[i] in anyoulist:
        continue
    else:
        anyoulist.append(anyou[i])

In [11]:
# 附录就是在重复之前的法条
del data['附录']

In [12]:
anyoulist

['侵害企业出资人权益纠纷',
 '意外伤害保险合同纠纷',
 '财产保险合同纠纷',
 '人身保险合同纠纷',
 '对外追收债权纠纷',
 '人寿保险合同纠纷',
 '责任保险合同纠纷',
 '保险费纠纷',
 '保险代理合同纠纷',
 '挂靠经营合同纠纷',
 '股权转让纠纷',
 '退伙纠纷',
 '与公司有关的纠纷',
 '保险人代位求偿权纠纷',
 '票据追索权纠纷',
 '申请破产清算',
 '票据返还请求权纠纷',
 '保险纠纷',
 '股东资格确认纠纷',
 '企业出售合同纠纷',
 '票据利益返还请求权纠纷',
 '财产损失保险合同纠纷',
 '确认票据无效纠纷',
 '破产债权确认纠纷',
 '保险经纪合同纠纷',
 '股东知情权纠纷',
 '损害公司利益责任纠纷',
 '股东损害公司债权人利益责任纠纷',
 '公司盈余分配纠纷',
 '清算责任纠纷',
 '与企业有关的纠纷',
 '公司决议纠纷',
 '联营合同纠纷',
 '国债交易纠纷',
 '与公司、证券、保险、票据等有关的民事纠纷',
 '企业承包经营合同纠纷',
 '企业租赁经营合同纠纷',
 '保证保险合同纠纷',
 '票据纠纷',
 '公司解散纠纷',
 '损害股东利益责任纠纷',
 '股东出资纠纷',
 '票据付款请求权纠纷',
 '请求变更公司登记纠纷',
 '信用证纠纷',
 '健康保险合同纠纷',
 '合伙企业纠纷',
 '公司债券交易纠纷',
 '公司决议撤销纠纷',
 '公司设立纠纷',
 '营业信托纠纷',
 '公司决议效力确认纠纷',
 '合伙企业财产份额转让纠纷',
 '证券发行纠纷',
 '证券交易代理合同纠纷',
 '证券虚假陈述责任纠纷',
 '公司增资纠纷',
 '入伙纠纷',
 '一般取回权纠纷',
 '公司证照返还纠纷',
 '信托纠纷',
 '欺诈客户责任纠纷',
 '职工破产债权确认纠纷',
 '企业兼并合同纠纷',
 '票据损害责任纠纷',
 '申请公司清算',
 '新增资本认购纠纷',
 '乡镇企业承包经营合同纠纷',
 '申请破产重整',
 '请求撤销个别清偿行为纠纷']

In [13]:
del data['庭审程序说明']

In [14]:
#处理庭审程序和文书类型
# 处理第三方
# 一审为1，二审为2,其他为0
thirdparty = []
for index in range(len(data)):        
    if "一审" in data.iloc[index]['庭审程序']:
        thirdparty.append(1)
    elif "二审" in data.iloc[index]['庭审程序']:
        thirdparty.append(2)
    else:
        thirdparty.append(0)
        
data['庭审程序分类'] = pd.DataFrame(thirdparty)
del data['庭审程序']

In [19]:
# 处理裁定书
thirdparty = []
for index in range(len(data)):        
    if "裁定书" in data.iloc[index]['文书类型']:
        thirdparty.append(1)
    elif "判决书" in data.iloc[index]['文书类型']:
        thirdparty.append(2)
    else:
        thirdparty.append(0)
        
data['文书'] = pd.DataFrame(thirdparty)
del data['文书类型']

In [23]:
laws = set(data['判决法律'])

In [34]:
# 处理结果
# 2 为其他
thirdparty = []
for index in range(len(data)):
    try:
        if "驳回" in data.iloc[index]['判决结果']:
            thirdparty.append(0)# 说明失败了
        else:
            thirdparty.append(1) # 认为成功了
    except:
        thirdparty.append(2)
        
data['是否成功'] = pd.DataFrame(thirdparty)
# del data['判决结果']

In [41]:
data = data.drop(1)

In [43]:
X = data[['是否为终审','原被告分类','第三方分类','庭审程序分类','文书']]
Y = data['是否成功']

In [47]:
from sklearn import tree

In [80]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X[:-200],Y[:-200])

In [81]:
(200 - np.sum(abs(clf.predict(X[-200:]) - Y[-200:])))/200

0.63